In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://de-zoomcamp.europe-west1-b.c.dataenginner-zoomcamp.internal:7077") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/28 15:49:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('/home/pedroh/dataengineer-zoomcamp2024/week6/code/data/pq/green/*/*')

24/02/28 15:50:18 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/02/28 15:50:33 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/02/28 15:50:48 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/02/28 15:51:03 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/02/28 15:51:18 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/02/28 15:51:33 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [5]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [6]:
df_yellow = spark.read.parquet('/home/pedroh/dataengineer-zoomcamp2024/week6/code/data/pq/yellow/*/*')

In [7]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [10]:
common_columns = ['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [11]:
from pyspark.sql import functions as F

In [12]:
df_green_final = df_green.select(common_columns).withColumn('service_type', F.lit('Green'))

In [13]:
df_yellow_final = df_yellow.select(common_columns).withColumn('service_type', F.lit('Yellow'))

In [14]:
df_trips_data = df_green_final.unionAll(df_yellow_final)

In [15]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       Green| 2304517|
|      Yellow|39649199|
+------------+--------+



In [16]:
df_trips_data.createOrReplaceTempView('trips_data')

In [17]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
     PULocationID AS revenue_zone,
     date_trunc('month', pickup_datetime) AS revenue_month,
     service_type, 

    -- Revenue calculation 
     ROUND(SUM(fare_amount),2) AS revenue_monthly_fare,
     ROUND(SUM(extra),2) AS revenue_monthly_extra,
     ROUND(SUM(mta_tax),2) AS revenue_monthly_mta_tax,
     ROUND(SUM(tip_amount),2) AS revenue_monthly_tip_amount,
     ROUND(SUM(tolls_amount),2) AS revenue_monthly_tolls_amount,
     ROUND(SUM(improvement_surcharge),2) AS revenue_monthly_improvement_surcharge,
     ROUND(SUM(total_amount),2) AS revenue_monthly_total_amount,
     ROUND(SUM(congestion_surcharge),2) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
     ROUND(AVG(passenger_count),2) AS avg_montly_passenger_count,
     ROUND(AVG(trip_distance),2) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

24/02/28 16:08:17 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
24/02/28 16:08:17 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:978)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [ ]:
df_result.write.mode("overwrite").parquet('/home/pedroh/dataengineer-zoomcamp2024/week6/code/data/report/revenue/')